In [1]:
import sys
sys.path.append('../')
import numpy as np
import pandas as pd
from grid_world.grid_world import GridWorld
from DMEIRL.DeepMEIRL_FC import DMEIRL
from grid_world.trajGen_grid_world import GridWorld_trajGen
from utils import utils

from grid_world.data_parser import DataParser
from grid_world.grid_world import GridWorld


import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2

no background image
no background image


In [2]:
parser = DataParser(width=10,height=10)
from PIL import Image
import os
from tqdm import tqdm
tracks_path = 'demo_label/train/'
file_names = os.listdir(tracks_path)
#read images
imgs = []
for file_name in file_names:
    imgs.append(Image.open(tracks_path + "/" + file_name))

#parse images
for i in tqdm(range(len(imgs))):
    parser.ParseEnvironmentFromImage(imgs[i],file_names[i].split('.')[0],save_path='demo_env&feature')

100%|██████████| 4/4 [00:00<00:00, 241.32it/s]


In [3]:
n = np.nan
state_mat = np.array([[ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                       [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                       [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                       [ 0, 0, 0, n, 0, 0, n, n, n, n],
                       [ 0, 0, 0, n, 0, 0, 0, 0, 0, 0],
                       [ 0, 0, 0, n, 0, 0, 0, 0, 0, 0],
                       [ 0, 0, 0, n, 0, 0, 8, 8, 8, 8],
                       [ 0, 0, 0, n, 0, 0, 8, 8, 8, 8],
                       [ 0, 0, 0, n, 0, 0, 8, 8, 8, 8],
                       [ 0, 0, 0, n, 0, 0, 8, 8, 8, 8],])

world_trajGen = GridWorld_trajGen(10,10,states_matrix=state_mat,
                                  features_folderPath='demo_env&feature/features_grid/0310_10x10',
                                  rewards_mul=[1,-2,-1,-2],
                                  init_states=list(range(0,10)),
                                  trans_prob=0.6,discount=0.95)
world_trajGen.ShowRewards(title='真实奖励')

In [4]:
world_trajGen.GenerateTrajectories(traj_count=500,traj_length=20,save=True)

100%|██████████| 500/500 [00:00<00:00, 4340.81it/s]


,m,trajs
0,1,"[(5, 1, 15), (15, 1, 25), (25, 1, 35), (35, 1,..."
1,2,"[(1, 4, 2), (2, 4, 3), (3, 4, 3), (3, 4, 4), (..."
2,3,"[(1, 4, 0), (0, 4, 10), (10, 2, 10), (10, 2, 0..."
3,4,"[(0, 4, 0), (0, 4, 0), (0, 4, 1), (1, 4, 2), (..."
4,5,"[(2, 4, 3), (3, 4, 4), (4, 1, 14), (14, 1, 24)..."
...,...,...
495,496,"[(0, 4, 0), (0, 4, 1), (1, 4, 1), (1, 4, 1), (..."
496,497,"[(9, 3, 9), (9, 3, 8), (8, 3, 9), (9, 3, 8), (..."
497,498,"[(6, 3, 5), (5, 1, 15), (15, 1, 25), (25, 1, 2..."
498,499,"[(8, 3, 7), (7, 3, 7), (7, 3, 6), (6, 3, 5), (..."


In [5]:
world_trajGen.ShowTrajs_Experts()

In [18]:
world = GridWorld(
    environments_img_folderPath='demo_label/train/',
    expert_trajs=world_trajGen.df_trajs_experts,
    width=10,
    height=10,
    active_all=False,
    trans_prob=0.6,
    discount=0.95,
    real_reward_mat = world_trajGen.real_rewards_matrix
)
world.ShowGridWorld_Count_log(title='轨迹计数分布(对数)')

parsing environments from folder:: 100%|██████████| 4/4 [00:00<00:00, 255.33it/s]


In [19]:
world.ShowDynamics(1)

In [20]:
world.ShowEnvironments()

In [21]:
world.ShowFeatures()

In [22]:
world.real_reward_arr.shape

(77,)

In [23]:
world.ShowGridWorld_Activated()

In [24]:
dme = DMEIRL(world,layers = (16,32,64,32,16),lr = 0.0001,log=utils.date+'(4)',log_dir='test_run')
rewards_result1,best_iter,rewards1 = dme.train(n_epochs=300,save = False,demo=True)

100%|██████████| 300/300 [24:04<00:00,  4.81s/it]


In [25]:
len(rewards_result1)

77

In [26]:
world.ShowRewardsResult(rewards1[-1],title='DMEIRL奖励复原结果')

In [27]:
world.ShowRewardsAnimation(rewards1,title='DMEIRL奖励复原动画')

In [28]:
dme.SaveModel('relu_model.pth')